### colab runtime 변경
- 코랩 런타임을 T4 GPU로 변경해주세요.
- 무료 버전 코랩을 사용할 경우, 하루 할당된 GPU가 제한되어 있어서 학습을 진행하다가 런타임이 끊길 수 있습니다.
- 로컬에 파이썬 설치가 가능하다면 로컬에서 진행하는 것을 추천드립니다.
- 로컬에서 진행할 경우엔 conda를 활용하여 가상환경을 세팅해주세요.

### mount google drive
- 코랩을 활용할 경우, 구글 드라이브에 있는 데이터를 사용하기 위해서 마운트 작업이 필요합니다.
- 로컬에서 진행할 경우엔 생략해주세요.

In [61]:
!git add "02_yolo_모델_학습_코드.ipynb"
!git commit -m "edit) using GPU with Pytorch(poop)" 
!git push origin main

fatal: '02_yolo_모델_학습_코드.ipynb' 경로명세가 어떤 파일과도 일치하지 않습니다
현재 브랜치 main
브랜치가 'origin/main'에 맞게 업데이트된 상태입니다.

커밋하도록 정하지 않은 변경 사항:
  (무엇을 커밋할지 바꾸려면 "git add <파일>..."을 사용하십시오)
  (use "git restore <file>..." to discard changes in working directory)
	수정함:        ../../.gitignore
	수정함:        ../../01_데이터셋_구축_코드.ipynb
	수정함:        ../../02_yolo_모델_학습_코드.ipynb

추적하지 않는 파일:
  (커밋할 사항에 포함하려면 "git add <파일>..."을 사용하십시오)
	../../01_데이터셋_구축_코드_박다영.ipynb.zip
	../../data.yaml

커밋할 변경 사항을 추가하지 않았습니다 ("git add" 및/또는 "git commit -a"를
사용하십시오)
Everything up-to-date


In [64]:
!git status  # 변경 사항 확인

현재 브랜치 main
브랜치가 'origin/main'에 맞게 업데이트된 상태입니다.

커밋하도록 정하지 않은 변경 사항:
  (무엇을 커밋할지 바꾸려면 "git add <파일>..."을 사용하십시오)
  (use "git restore <file>..." to discard changes in working directory)
	수정함:        ../../.gitignore
	수정함:        ../../01_데이터셋_구축_코드.ipynb
	수정함:        ../../02_yolo_모델_학습_코드.ipynb

추적하지 않는 파일:
  (커밋할 사항에 포함하려면 "git add <파일>..."을 사용하십시오)
	../../01_데이터셋_구축_코드_박다영.ipynb.zip
	../../data.yaml

커밋할 변경 사항을 추가하지 않았습니다 ("git add" 및/또는 "git commit -a"를
사용하십시오)


### Import Libraray

In [3]:
import os
import cv2
import glob
import shutil
import warnings
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
#!pip install --upgrade jupyter ipywidgets

In [4]:
warnings.filterwarnings(action='ignore')

In [5]:
# 현재 실행 파일 경로 확인
os.getcwd()

'/Users/tasha/Desktop/comento/mywork'

### yolov5s 모델 설치 및 환경 설정

- 모델을 설치할 경로에 cd 명령어를 활용하여 들어가주세요

In [12]:
# 폴더 생성
#!mkdir -p /Users/tasha/Desktop/comento/mywork/models
!mkdir -p /Users/tasha/Desktop/comento/mywork/models/yolov5
!mkdir -p /Users/tasha/Desktop/comento/mywork/models/yolov8
!mkdir -p /Users/tasha/Desktop/comento/mywork/models/yolov11

#git ignore 추가 
#!echo "models/" >> /Users/tasha/Desktop/comento/mywork/.gitignore

In [13]:
cd /Users/tasha/Desktop/comento/mywork/models/yolov5

/Users/tasha/Desktop/comento/mywork/models/yolov5


In [14]:
# 현재 파일 경로 확인 (원하는 경로에 들어와있는지 확인)
os.getcwd()

'/Users/tasha/Desktop/comento/mywork/models/yolov5'

In [15]:
# git clone을 통해 yolov5s 모델 설치
!git clone https://github.com/ultralytics/yolov5.git

'yolov5'에 복제합니다...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
오브젝트를 받는 중: 100% (17274/17274), 16.12 MiB | 16.50 MiB/s, 완료.
델타를 알아내는 중: 100% (11862/11862), 완료.


### yolov5s 전이학습
- 전이학습을 진행할 때 사용되는 batch와 epoch에 대해서 정의도 같이 공부하면 좋을 것 같습니다.
- 전이학습 후에 나오는 최종모델파일은 마지막줄에 출력되는 "Results saved to yolov5/runs/train/yolov5s_results" 에 저장됩니다.

In [51]:
#!pip install --upgrade Pillow
#!pip install torch torchvision torchaudio 
#!pip install pandas
#!pip install numpy==2.1.1 # yolov5s에 맞춤
#!pip install --upgrade scipy
#!pip install --upgrade seaborn
#!pip install tensorboard
#!pip install tensorflow

In [43]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
# MPS는 M1 GPU에서 Metal API를 사용하는 PyTorch 백엔드입니다.
print(torch.backends.mps.is_available())  # M1 GPU(Metal)가 사용 가능한지 확인

#torch는 gpu를 기본값으로 쓰도록 설정 
#device = torch.device("mps")

2.6.0
True


In [30]:
#캐시 비우기
#!rm -f /Users/tasha/Desktop/comento/dataset/datasets/labels/train.cache

In [44]:
import timeit
start = timeit.default_timer()

!python yolov5/train.py --img 224 --batch 20 --epochs 10 --data /Users/tasha/Desktop/comento/dataset/datasets/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_results 

end = timeit.default_timer()
print(end-start)

train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/Users/tasha/Desktop/comento/dataset/datasets/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=20, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.12.9 torch-2.6.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warm

In [52]:
# 텐서보드로 학습률 상태 바로 확인 가능 
!tensorboard --logdir yolov5/runs/train

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### 전이학습 결과 확인
- 1주차 과제에서 생성한 testdataset을 기반으로 모델 성능을 평가합니다.

In [ ]:
!python yolov5/val.py --img 224 --batch 60 --data datasets_yolo/datasets.yaml --weights yolov5/runs/train/yolov5s_results/weights/best.pt

val: data=datasets_yolo/datasets.yaml, weights=['yolov5/runs/train/yolov5s_results4/weights/best.pt'], batch_size=60, imgsz=224, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/comento/1/datasets_yolo/labels/val.cache... 399 images, 0 backgrounds, 0 corrupt: 100% 399/399 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:43<00:00,  6.21s/it]
                   all        399       1072          1          1      0.995      0.901
Speed: 0.0ms pre-process, 1.1ms inference, 7.3ms NMS per imag

### 추론(inference) 진행
- 학습에 사용되지 않은 이미지를 한 장씩 넣어서 추론 결과 확인해보기
- 추론 속도와 추론 결과 이미지 확인

In [ ]:
# 필요할 경우 설치
# !pip install ultralytics

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# 학습한 모델 불러오기
model = torch.hub.load('yolov5', 'custom', path='yolov5/runs/train/yolov5s_results4/weights/best.pt', source='local')
model.conf = 0.5

YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
# 추론을 진행할 이미지 불러오기
test_img_path = 'Enter inference image path'
img = Image.open(test_img_path)

In [ ]:
# 추론 진행
results = model(img, size=224)

/content/drive/MyDrive/comento/1/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# 추론 결과 확인하기
print(results.pandas().xyxy[0])

# Save or display results
results.show()
plt.imshow(results.render()[0])
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.